In [ ]:
!pip install contextualized-topic-models==2.3.0

In [1]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
import nltk

In [5]:
import pandas as pd
data=pd.read_csv('/content/_bstractive_concatenated_data.csv')


In [7]:
### Using LDA

In [8]:
import re
# Remove punctuation
data['abstractive_summarization_data_processed'] = \
data['abstractive_summarization_data'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
data['abstractive_summarization_data_processed']  = \
data['abstractive_summarization_data_processed'] .map(lambda x: x.lower())
# Print out the first rows of papers
data['abstractive_summarization_data_processed'] .head()


0    proof the mainstream media is manipulating the...
1    charity: clinton foundation distributed “water...
2    a hillary clinton administration may be entire...
3    trump’s latest campaign promise may be his mos...
4    website is down for maintenancewebsite is down...
Name: abstractive_summarization_data_processed, dtype: object

In [9]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = data.abstractive_summarization_data_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])


[nltk_data] Downloading package stopwords to /root/nltk_data...


['proof', 'mainstream', 'media', 'manipulating', 'election', 'taking', 'bill', 'clinton', 'contexti', 'woke', 'morning', 'find', 'variation', 'headline', 'splashed', 'news', 'feed', 'bill', 'clinton', 'natural', 'foundation', 'donors', 'seek', 'favors', 'google', 'naturally', 'reaction', 'oh', 'bill', 'clinton']


[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 3), (1, 1), (2, 3), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)]


In [35]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=20, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=20,
                                       per_word_topics=True)
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.096*"clinton" + 0.076*"hillary" + 0.040*"trump" + 0.021*"first" + 0.019*"new" + 0.017*"cnn" + 0.017*"story" + 0.016*"donald" + 0.015*"debate" + 0.014*"asked"'), (1, '0.027*"news" + 0.017*"obama" + 0.017*"country" + 0.015*"trump" + 0.014*"stop" + 0.014*"washington" + 0.013*"new" + 0.013*"cnn" + 0.012*"serve" + 0.012*"frisk"'), (2, '0.037*"clinton" + 0.026*"foundation" + 0.019*"bill" + 0.018*"drugs" + 0.017*"aids" + 0.016*"sub" + 0.016*"watered" + 0.016*"distributed" + 0.016*"saharan" + 0.016*"africa"'), (3, '0.029*"trump" + 0.025*"face" + 0.024*"said" + 0.023*"america" + 0.019*"prime" + 0.019*"minister" + 0.019*"presidential" + 0.018*"relationships" + 0.018*"week" + 0.018*"york"'), (4, '0.031*"aleppo" + 0.031*"syrian" + 0.031*"area" + 0.020*"northern" + 0.019*"chief" + 0.018*"leaves" + 0.018*"sept" + 0.018*"forward" + 0.017*"going" + 0.017*"slammed"'), (5, '0.039*"city" + 0.036*"black" + 0.032*"suspect" + 0.027*"new" + 0.024*"breaking" + 0.021*"one" + 0.020*"shooting" + 0.016*"w

In [45]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.096*"clinton" + 0.076*"hillary" + 0.040*"trump" + 0.021*"first" + '
  '0.019*"new" + 0.017*"cnn" + 0.017*"story" + 0.016*"donald" + 0.015*"debate" '
  '+ 0.014*"asked"'),
 (1,
  '0.027*"news" + 0.017*"obama" + 0.017*"country" + 0.015*"trump" + '
  '0.014*"stop" + 0.014*"washington" + 0.013*"new" + 0.013*"cnn" + '
  '0.012*"serve" + 0.012*"frisk"'),
 (2,
  '0.037*"clinton" + 0.026*"foundation" + 0.019*"bill" + 0.018*"drugs" + '
  '0.017*"aids" + 0.016*"sub" + 0.016*"watered" + 0.016*"distributed" + '
  '0.016*"saharan" + 0.016*"africa"'),
 (3,
  '0.029*"trump" + 0.025*"face" + 0.024*"said" + 0.023*"america" + '
  '0.019*"prime" + 0.019*"minister" + 0.019*"presidential" + '
  '0.018*"relationships" + 0.018*"week" + 0.018*"york"'),
 (4,
  '0.031*"aleppo" + 0.031*"syrian" + 0.031*"area" + 0.020*"northern" + '
  '0.019*"chief" + 0.018*"leaves" + 0.018*"sept" + 0.018*"forward" + '
  '0.017*"going" + 0.017*"slammed"'),
 (5,
  '0.039*"city" + 0.036*"black" + 0.032*"suspect" + 0.027*"

In [22]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


Coherence Score:  nan


In [44]:
data[:3]


['proof the mainstream media is manipulating the election by taking bill clinton out of contexti woke up this morning to find a variation of this headline splashed all over my news feed:\n\nbill clinton: ‘natural’ for foundation donors to seek favors\n\nhere’s google:\n\nnaturally my reaction was “oh s**t what did bill clinton do to damage his wife’s campaign now”',
 'charity: clinton foundation distributed “watered-down” aids drugs to sub-saharan africa – eagle risingformer president bill clinton and his clinton health access initiative (chai) distributed “watered-down” hiv/aids drugs to patients in sub-saharan africa and “likely increased” the risks of morbidity and mortality according to a draft congressional report obtained by the daily caller news foundation',
 'a hillary clinton administration may be entirely run by a figurehead – eagle risingafter collapsing just before trying to step into her van after being removed from the 9-11 ceremony in new york city hillary clinton emerge

In [6]:
pip install bertopic

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.0 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp310-cp310-linux_x86_64.whl size=3541452 sha256=13ad2709f79f575f5099af1ae68076947a703521a0a6f652774086119d7de32a
  Stored in dir

In [10]:
from bertopic import BERTopic
#from sklearn.datasets import fetch_20newsgroups
 
#docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
#sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
topic_model = BERTopic(nr_topics=10,top_n_words=10)
topics, probs = topic_model.fit_transform(data)

In [11]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,47,-1_to_the_in_and
1,0,39,0_charlotte_police_the_in
2,1,11,1_joe_mccain_sheriff_arpaio
3,2,182,2_the_trump_clinton_to
4,3,49,3_obama_to_the_be
5,4,43,4_new_news_in_khan
6,5,27,5_obama_the_isis_of
7,6,11,6_wall_construction_great_migrants
8,7,13,7_jr_tweet_syrian_trump


In [1]:
topic_model.transform(data)

NameError: ignored

In [15]:
dictionary_of_topics=topic_model.get_representative_docs()

In [40]:
for i in topic_model.get_topic(3):
  print(i[0])

obama
to
the
be
trump
for
on
it
and
in


In [23]:
dictionary_of_topics.keys()

dict_keys([-1, 0, 1, 2, 3, 4, 5, 6, 7])

In [25]:
t=topic_model.transform(data)

In [33]:
topics_generated=t[0]


In [31]:
topics_on_abstracted=pd.read_csv('/content/_bstractive_concatenated_data.csv')

In [34]:
topics_on_abstracted['topics']=topics_generated

In [35]:
topics_on_abstracted.to_csv("topics_on_abstracted.csv")

In [41]:
train_data=['the trump clinton to of hillary in is and donald',
            'charlotte police the in black of to for and after',
            'obama to the be trump for on it and in']

In [42]:
train_data

['the trump clinton to of hillary in is and donald',
 'charlotte police the in black of to for and after',
 'obama to the be trump for on it and in']

In [43]:
train_labels=[[0,1],[0,1],[1,0]]

In [47]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"



SyntaxError: ignored

In [48]:
!pip install lambeq

NotImplementedError: ignored